In [1]:
import pulp #pulp will be used to produce the problem
import numpy as np

In [2]:
#i is the row and j is the column
c = np.array([[0,104.69002,38.832976,73.925638,38.209946],
              [104.6900186,0,66.60330322,43.737855,66.483081],
              [38.83297568,66.603303,0,36.674242,5.6568542],
              [73.92563831,43.73785546,36.67424164,0,40.112342],
              [38.209946,66.483081,5.656854249,40.11234224,0]])

In [3]:
#create the model
model = pulp.LpProblem("part3",pulp.LpMinimize)
#set the Xij boolean variables 
x = pulp.LpVariable.dicts("X",((i, j) for i in range(5) for j in range(5)),
						  lowBound=0,cat='Binary')

In [6]:
#the inbound/outbound constraint 
for i in range(5):
    model += pulp.lpSum([x[i,j] for j in range(5) if i != j]) == 1
for i in range(5):
    model += pulp.lpSum([x[i,i]]) == 0
#subtours (2 nodes)
for i in range(5):
    for j in range(5):
        if i != j:
            model += pulp.lpSum([x[i,j]+x[j,i]]) <= 1
#subtours (3 nodes)
for i in range(5):
    for j in range(5):
        for k in range(5):
            if (i != j and i!=k and j != k):
                model += pulp.lpSum([x[i,j]+x[j,k]+x[k,i]]) <= 2
#subtours (4 nodes)
for i in range(5):
    for j in range(5):
        for k in range(5):
            for l in range(5):
                if (i != j and i!=k and i != l and
                    j != k and j != l and k != l):
                    model += pulp.lpSum([x[i,j]+x[j,k]+x[k,l]+x[l,i]]) <= 3

In [7]:
#all the cost times x is the minimum
model += pulp.lpSum([c[i,j]*x[i,j] for i in range(5) 
					 for j in range(5) if i!=j])

C:\Users\86185\anaconda3\lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [8]:
model.solve()
#create the solution matrix,where X is the R matrix and Y the suppliers matrix
X = np.zeros(5*5)
i = 0
#assign the variables to the matrix
for variable in model.variables():
    if i<25:
        X[i]= variable.varValue
    i += 1
X = np.reshape(X,(5,5))
print(model)
print('X =\n',X)
print('Minimum cost:',pulp.value(model.objective))

part3:
MINIMIZE
104.69002*X_(0,_1) + 38.832976*X_(0,_2) + 73.925638*X_(0,_3) + 38.209946*X_(0,_4) + 104.6900186*X_(1,_0) + 66.60330322*X_(1,_2) + 43.737855*X_(1,_3) + 66.483081*X_(1,_4) + 38.83297568*X_(2,_0) + 66.603303*X_(2,_1) + 36.674242*X_(2,_3) + 5.6568542*X_(2,_4) + 73.92563831*X_(3,_0) + 43.73785546*X_(3,_1) + 36.67424164*X_(3,_2) + 40.112342*X_(3,_4) + 38.209946*X_(4,_0) + 66.483081*X_(4,_1) + 5.656854249*X_(4,_2) + 40.11234224*X_(4,_3) + 0.0
SUBJECT TO
_C1: X_(0,_1) + X_(0,_2) + X_(0,_3) + X_(0,_4) = 1

_C2: X_(1,_0) + X_(1,_2) + X_(1,_3) + X_(1,_4) = 1

_C3: X_(2,_0) + X_(2,_1) + X_(2,_3) + X_(2,_4) = 1

_C4: X_(3,_0) + X_(3,_1) + X_(3,_2) + X_(3,_4) = 1

_C5: X_(4,_0) + X_(4,_1) + X_(4,_2) + X_(4,_3) = 1

_C6: X_(0,_0) = 0

_C7: X_(1,_1) = 0

_C8: X_(2,_2) = 0

_C9: X_(3,_3) = 0

_C10: X_(4,_4) = 0

_C11: X_(0,_1) + X_(1,_0) <= 1

_C12: X_(0,_2) + X_(2,_0) <= 1

_C13: X_(0,_3) + X_(3,_0) <= 1

_C14: X_(0,_4) + X_(4,_0) <= 1

_C15: X_(0,_1) + X_(1,_0) <= 1

_C16: X_(1,_2) + 